In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
from ultralytics import YOLO
import glob

model = YOLO("./models/yolo11n-obb-custom.pt")

image_files = glob.glob("./predict/image_*")

for file in image_files:
    image = cv2.imread(file)

    results = model(image, agnostic_nms=True, iou=0.3)

    class_names = {0: "Plate1", 1: "Plate3"}
    class_colors = {0: (255, 0, 0), 1: (0, 0, 255)}

    res = results[0].obb

    for box in res:
        coords = box.xyxyxyxy.cpu().numpy().astype(np.int32)

        class_id = int(box.cls.cpu())
        confidence = box.conf.cpu().numpy()

        if confidence[0] < 0.5:
            continue

        color = class_colors.get(class_id, (0, 255, 0))

        cv2.drawContours(image, [coords], 0, color, 1)

        class_name = class_names.get(class_id, "Unknown")
        confidence_text = f"{class_name} ({confidence[0]:.2f})"

        cv2.putText(image, confidence_text, 
                    (coords[0][0][0], coords[0][0][1] - 10), 
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    1.5, color, 1)

    plt.figure()
    plt.figure(figsize=(40,10))
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.axis("off")
    plt.show()